# Analítica Avanzada de Datos.
---

# Clasificación

Las técnicas de aprendizaje automático *supervisado* consisten en entrenar un modelo para que opere sobre un conjunto de *características* y prediga una etiqueta utilizando un conjunto de datos que incluye algunos valores de etiqueta ya conocidos. Puedes pensar en esta función así, en la que ***y*** representa la etiqueta que queremos predecir y ***X*** representa el vector de características que el modelo utiliza para predecirla.

$$y = f([x_1, x_2, x_3, ...])$$

La *clasificación* es una forma de aprendizaje automático supervisado en el que se entrena un modelo para utilizar las características (los valores ***x*** de nuestra función) para predecir una etiqueta (***y***) que calcula la probabilidad de que el caso observado pertenezca a cada una de las clases posibles y predice una etiqueta adecuada. La forma más sencilla de clasificación es la clasificación binaria, en la que la etiqueta es 0 o 1, representando una de dos clases; por ejemplo, "Verdadero" o "Falso"; "Interno" o "Externo"; "Rentable" o "No rentable"; etc.

## Clasificación binaria

En este notebook, nos centraremos en un ejemplo de *clasificación binaria*, donde el modelo debe predecir una etiqueta que pertenece a una de dos clases. Entrenaremos un clasificador binario para predecir si un paciente debe o no someterse a una prueba de diabetes basándonos en algunos datos médicos.


### Explorar los datos
Ejecuta la siguiente celda para cargar un archivo CSV de datos de patentes en un marco de datos de **Pandas**:

> **Cita**: El dataset sobre diabetes utilizado en este ejercicio se basa en datos recogidos originalmente por el "National Institute of Diabetes and Digestive and Kidney Diseases".

In [ ]:
import pandas as pd

# cargamos el dataset de prueba
diabetes = pd.read_csv('diabetes.csv')
diabetes.head()

Estos datos consisten en información diagnóstica sobre algunos pacientes a los que se les ha realizado la prueba de la diabetes. Desplácese a la derecha si es necesario, y observe que la última columna del conjunto de datos (**Diabético**) contiene el valor ***0*** para los pacientes que dieron negativo en la prueba de la diabetes, y ***1*** para los pacientes que dieron positivo. La mayoría de las demás columnas (Embarazos, Glucosa plasmática, Presión arterial diastólica, etc.) son las características que utilizaremos para predecir la etiqueta **Diabético**.

Separemos las características de las etiquetas: llamaremos ***X*** a las características e ***y*** a la etiqueta:

In [ ]:
# Funciones y etiquetas separadas
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
label = 'Diabetic'
X, y = diabetes[features].values, diabetes[label].values

for n in range(0,4):
    print("Patient", str(n+1), "\n  Features:",list(X[n]), "\n  Label:", y[n])

Ahora comparemos las distribuciones de características para cada valor de etiqueta.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
for col in features:
    diabetes.boxplot(column=col, by='Diabetic', figsize=(6,6))
    plt.title(col)
plt.show()

Para algunas de las características, hay una diferencia notable en la distribución para cada valor de etiqueta. En particular, **Pregnancies** y **Age**  muestran distribuciones notablemente diferentes para los pacientes diabéticos que para los no diabéticos. Estas características pueden ayudar a predecir si un paciente es diabético o no.


### Dividir los datos¶

Nuestro dataset incluye valores conocidos para la etiqueta, por lo que podemos utilizarlo para entrenar un clasificador de forma que encuentre una relación estadística entre las características y el valor de la etiqueta; pero ¿cómo sabremos si nuestro modelo es bueno? Podemos aprovechar que tenemos un gran conjunto de datos con valores de etiqueta conocidos, utilizar sólo una parte para entrenar el modelo y retener otra parte para probar el modelo entrenado, lo que nos permite comparar las etiquetas predichas con las etiquetas ya conocidas del conjunto de prueba.

En Python, el paquete **scikit-learn** contiene un gran número de funciones que podemos utilizar para construir un modelo de aprendizaje automático, incluida una función **train_test_split** que garantiza que obtengamos una división estadísticamente aleatoria de los datos de entrenamiento y de prueba. Utilizaremos esta función para dividir los datos en un **70%** para el entrenamiento y retener un **30%** para las pruebas.

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos entre un 70% y un 30% en conjunto de entrenamiento y conjunto de prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training cases: %d\nTest cases: %d' % (X_train.shape[0], X_test.shape[0]))

### Entrenar y evaluar un modelo de clasificación binaria

Ahora, ya estamos listos para entrenar nuestro modelo ajustando las características de entrenamiento (**X_train**) a las etiquetas de entrenamiento (**y_train**). Hay varios algoritmos que podemos utilizar para entrenar el modelo. En este ejemplo, utilizaremos la regresión logística, que (a pesar de su nombre) es un algoritmo bien establecido para la clasificación. Además de las características y etiquetas de entrenamiento, tendremos que establecer un parámetro de *regularización*. Esto se utiliza para contrarrestar cualquier sesgo en la muestra, y ayudar a que el modelo generalice bien evitando el sobreajuste del modelo a los datos de entrenamiento.


> **Nota**: Los parámetros de los algoritmos de aprendizaje automático suelen denominarse *hiperparámetros* (para un científico de datos, los *parámetros* son valores de los propios datos; los *hiperparámetros* se definen externamente a partir de los datos).

In [ ]:
# Entrenar el modelo
from sklearn.linear_model import LogisticRegression

# Fijar la tasa de regularización
reg = 0.01

# entrenar un modelo de regresión logística en el conjunto de entrenamiento
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)
print (model)

Ahora que hemos entrenado el modelo con los datos de entrenamiento, podemos utilizar los datos de prueba que retuvimos para evaluar lo bien que predice. Una vez más, **scikit-learn** puede ayudarnos a hacerlo. Empecemos utilizando el modelo para predecir las etiquetas de nuestro conjunto de prueba y comparemos las etiquetas predichas con las etiquetas conocidas:

In [ ]:
predictions = model.predict(X_test)
print('Predicted labels: ', predictions)
print('Actual labels:    ' ,y_test)

Las matrices de etiquetas son demasiado largas para mostrarlas en la salida del notebook, por lo que sólo podemos comparar unos pocos valores. Incluso si imprimimos todas las etiquetas predichas y reales, hay demasiadas como para que esta sea una forma sensata de evaluar el modelo. Afortunadamente, **scikit-learn** proporciona algunas métricas que podemos utilizar para evaluar el modelo.

Lo más obvio que podemos hacer es comprobar el *accuracy* de las predicciones: en términos sencillos, ¿qué proporción de las etiquetas predijo correctamente el modelo?

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_test, predictions))

El *accuracy* se devuelve como un valor decimal: un valor de 1.0 significaría que el modelo acertó el 100% de las predicciones, mientras que una precisión de 0.0 es, bueno, eso no es muy útil

En este notebook hemos preparado nuestros datos dividiéndolos en conjuntos de datos de prueba y de entrenamiento, y aplicado la regresión logística. 

Nuestro modelo fue capaz de predecir si los pacientes tenían diabetes con una precisión que parece razonable. Pero, ¿es suficiente? En el notebook ***02.Clasificacion_Metricas*** veremos alternativas a la precisión que pueden ser mucho más útiles en el aprendizaje automático.
